In [1]:
%matplotlib widget

In [2]:
from functools import partial

In [3]:
from libschrodinger.crank_nicolson_2d import *

In [4]:
from libschrodinger.potentials import *

In [5]:
from libschrodinger.campaigns import *

In [2]:
import numpy as np

In [7]:
import pandas as pd

In [8]:
from pathlib import Path

In [7]:
from typing import Dict, List, Tuple

In [10]:
from scipy import special

In [11]:
caseName : str = "bouncing_plate_0"

In [12]:
class PropigationDirection(Enum): 
    Forward : float = 1
    Backward : float = -1

In [14]:
math = np
spatialStep : float = .01
length : float = 1
temporalStep : float = (spatialStep ** 2) / 4
courantNumber : float = 1.0
pointCount : int = int(length / spatialStep)
potentialHeight : float = 200
preRegionLength = .3
preRegionScalar = 10
packetStandardDeviation : float = preRegionLength / preRegionScalar
packetStartX : float = .5#1 - ((preRegionScalar / 2) * packetStandardDeviation)
packetStartY : float = .5
angularFrequency : float = 50.0
speedOfLight : float = 1
#packetWaveNumber : float = 13 * np.pi

barrierCount : int = 3
postRegionLength = .3
intraRegionLength : float = 1 - (postRegionLength + preRegionLength)
barrierWidth : float = intraRegionLength / barrierCount
constantPotentialLengthRatios : List[float] = [postRegionLength] \
        + ([barrierWidth] * barrierCount) \
        + [preRegionLength]
print("Point Count: ", pointCount)
print("Post Region Length: ", postRegionLength)
print("Intra Region Length: ", intraRegionLength)
print("Pre Region Length: ", preRegionLength)
print("Barrier Length: ", barrierWidth)
print("Total Length: ", intraRegionLength + postRegionLength + preRegionLength)
print("Total Length Ratios: ", np.sum(constantPotentialLengthRatios))
frameCount : int = 100 #650
backend = "numpy"

Point Count:  100
Post Region Length:  0.3
Intra Region Length:  0.4
Pre Region Length:  0.3
Barrier Length:  0.13333333333333333
Total Length:  1.0
Total Length Ratios:  1.0


In [15]:
wavePacketFunction = lambda position : makeNonDispersiveWavePacket(
            position, 
            packetStartX * length, 
            packetStartY * length, 
            angularFrequency, 
            speedOfLight, 
            packetStandardDeviation
        )

In [16]:
potentials : List[List[float]] = [
        [0, 0, 0, 0, 0]
    ]

In [17]:
profiles : List[SimulationProfile] = constantSimulationProfiles(
        wavePacketFunction, 
        spatialStep, 
        temporalStep, 
        length, 
        constantPotentialLengthRatios, 
        potentials, 
        False, 
        pointCount, 
        False, 
        edgeBound = True
    )

In [18]:
constantPotentialLengthRatios

[0.3, 0.13333333333333333, 0.13333333333333333, 0.13333333333333333, 0.3]

In [19]:
allData, simulations = recordConstantRegionSimulations(
        profiles, 
        frameCount, 
        caseName, 
        constantPotentialLengthRatios, 
        True, 
        constantRegionLabels = ["Post", "3rdStair", "2ndStair", "1stStair", "Pre"], 
        showBar = True, 
        showFPS = True, 
        showTotalTime = True, 
        discardSimulations = False
    )

[====================================================================================================]
[---------------------------------------------------------------------------------------------------]
Total Time:  8.435457311999926
Frames Per Second:  12.589993403600904
Simulation 0 is done, processing probabilities.
Simulation 0: done processing probabilities.
Simulation 0: logging.
Saving Video of airy10
Saving Video of Post
Saving Video of 3rdStair
Saving Video of 2ndStair
Saving Video of 1stStair
Saving Video of Pre
Done logging airy10
Producing Simulation final output CSV


In [20]:
allData["packetStartX"] = packetStartX
allData["packetStartY"] = packetStartY
allData["initialWaveNumber"] = "NULL"#packetWaveNumber
allData["standardDeviation"] = packetStandardDeviation

In [21]:
pd.DataFrame(allData).to_csv(str(Path.cwd() / caseName / (caseName + ".csv")))